In [ ]:
import numpy as np
import math
import torch             # torch基础库
import torch.nn as nn    # torch神经网络库
from torch.utils.data import DataLoader, Dataset
from early_stopping import EarlyStopping
import os, gc
import random 
import subprocess
import rpy2.robjects as robjects



def get_gpu_memory(device_id):
    """
    Retrieve the memory usage of a specific GPU.

    Parameters:
    device_id: ID of the GPU device to query.

    Returns: A tuple (memory_used, memory_total) in MB, or (None, None) if an error occurs.
    """
    try:
        output = subprocess.check_output(["nvidia-smi", "--id={}".format(device_id), "--query-gpu=memory.used,memory.total", "--format=csv,nounits,noheader"])
        memory_used, memory_total = map(int, output.decode("utf-8").strip().split("\n")[0].split(","))
        return memory_used, memory_total
    except Exception as e:
        print(e)
        return None, None

def get_free_gpu():
    """
    Find the GPU with the most available memory and return its device ID.
    
    Returns:
    torch.device or None: The device with the most free memory if available, otherwise None.
    """
    device_ids = list(range(torch.cuda.device_count()))
    memory_usages = []
    for device_id in device_ids:
        memory_used, memory_total = get_gpu_memory(device_id)
        if memory_used is not None and memory_total is not None:
            memory_free = memory_total - memory_used
            memory_usages.append((device_id, memory_free))
        print(memory_total,memory_usages)
    if len(memory_usages) > 0:
        best_device_id = sorted(memory_usages, key=lambda x: x[1])[len(device_ids)-1][0]
        device = torch.device(f"cuda:{best_device_id}")
        return device
    else:
        return None


class Args:
    """
    A class to store tuning parameters for model training.
    
    Attributes:
    batch_size: batch size.
    lr: Learning rate for the optimizer.
    nepoch: Total number of epochs for training.
    patience: Number of epochs to wait for improvement before stopping early.
    wide: Width of the model, representing the number of units in each layer.
    depth: Depth of the model, representing the number of layers.
    n_train (int): Sample size.
    m_train (int): Sampling frequency.
    biaoji (str): A unique identifier to aovid confusion.
    """
    def __init__(self, batch_size=10, lr =0.001, nepoch = 200, patience = 10, wide = 100, depth = 5, n_train=1, m_train=1) -> None:
        self.batch_size = batch_size
        self.lr = lr
        self.nepoch = nepoch 
        self.patience = patience 
        self.wide = wide 
        self.depth = depth 
        self.biaoji = "wide" + str(wide) + "depth" + str(depth) + "n" + str(n_train) + "m" + str(m_train)
        self.n_train = n_train
        self.m_train = m_train


class EarlyStopping():
    """
    Early stopping utility to halt training when validation loss does not improve.
    
    Attributes:
    save_path: Path where model checkpoints are saved.
    patience: Number of epochs to wait for improvement before stopping.
    verbose: If True, prints validation loss improvements.
    delta: Minimum change in validation loss to qualify as an improvement.
    counter: Tracks epochs without improvement.
    best_score: Best score achieved on validation loss (initialized to None).
    early_stop: Flag to indicate if training should be stopped.
    val_loss_min: Tracks the minimum validation loss seen so far.
    """
    def __init__(self, save_path, args, verbose=False, delta=0):
        self.save_path = save_path 
        self.patience = args.patience 
        self.verbose = verbose 
        self.counter = 0 
        self.best_score = None 
        self.early_stop = False 
        self.val_loss_min = np.Inf 
        self.delta = delta 

    def __call__(self, model, train_loss, valid_loss, args, seed):

        score = -valid_loss 

        if self.best_score is None: 
            self.best_score = score 
            self.save_checkpoint(model, train_loss, valid_loss, args, seed) 
        elif score < self.best_score + self.delta: 
            self.counter += 1 
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}') 
            if self.counter >= self.patience: 
                self.early_stop = True 
        else:
            self.best_score = score
            self.save_checkpoint(model, train_loss, valid_loss,  args, seed)
            self.counter = 0

    def save_checkpoint(self, model, train_loss, valid_loss,  args, seed):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {valid_loss:.6f}).  Saving model ...') 
        torch.save(model, os.path.join(self.save_path, 'best' + str(seed) + args.biaoji +'network.pth') )	
        torch.save(train_loss, os.path.join(self.save_path, 'best'+ str(seed) + args.biaoji +'train_loss.pth')) 
        torch.save(valid_loss, os.path.join(self.save_path, 'best'+ str(seed) + args.biaoji +'valid_loss.pth')) 

        self.val_loss_min = valid_loss
    


class Dataset_repeatedmeasurement(Dataset): 
    """
    A custom dataset class for handling repeated measurement data.
    
    Attributes:
    x: Input data of features.
    y: Target labels corresponding to the input data.
    
    Methods:
    __len__: Returns the total number of samples in the dataset.
    __getitem__: Retrieves a single sample, returning a dictionary with 'x' and 'y' keys.
    """
    def __init__(self, x, y) -> None: 
        """
        Initialize the dataset with input data and corresponding labels.
        """
        super().__init__()
        self.x = x 
        self.y = y 

    def __len__(self) -> int: 
        """
        Return the number of samples in the dataset.
        """
        return len(self.x) 
    
    def __getitem__(self, index): 
        """
        Retrieve a sample from the dataset at the specified index.
        """
        return self.x[index], self.y[index]



class happynet(nn.Module):
    """
    A flexible neural network with a customizable depth.
    
    Parameters:
    n_feature: Dimension of input.
    n_hidden: Number of units in each hidden layer.
    n_output: Number of output units.
    n_layer: the number of layers (supports 3 to 10 layers). 
             (n_layer-1) hidden layers
    
    Methods:
    forward(x): Forward pass through the network.
    """
    def __init__(self, n_feature, n_hidden, n_output, n_layer): 
        super().__init__()
        if n_layer == 3: 
            self.net = nn.Sequential(
                nn.Linear(n_feature, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_output), 
            )
        elif n_layer == 2: 
            self.net = nn.Sequential(
                nn.Linear(n_feature, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_output), 
            )    
        elif n_layer == 4: 
            self.net = nn.Sequential(
                nn.Linear(n_feature, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_output), 
            )
        elif n_layer == 5: 
            self.net = nn.Sequential(
                nn.Linear(n_feature, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden),  
                nn.ReLU(),
                nn.Linear(n_hidden, n_output),
            )
        elif n_layer == 6: 
            self.net = nn.Sequential(
                nn.Linear(n_feature, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden),  
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_output),
            )
        elif n_layer == 7: 
            self.net = nn.Sequential(
                nn.Linear(n_feature, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden),  
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden),
                nn.ReLU(),
                nn.Linear(n_hidden, n_output),
            )
        elif n_layer == 8: 
            self.net = nn.Sequential(
                nn.Linear(n_feature, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden),  
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden),
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden),
                nn.ReLU(),
                nn.Linear(n_hidden, n_output),
            )
        elif n_layer == 9: 
            self.net = nn.Sequential(
                nn.Linear(n_feature, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden),  
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden),
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden),
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden),
                nn.ReLU(),
                nn.Linear(n_hidden, n_output),
            )
        elif n_layer == 10: 
            self.net = nn.Sequential(
                nn.Linear(n_feature, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden),  
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden), 
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden),
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden),
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden),
                nn.ReLU(),
                nn.Linear(n_hidden, n_hidden),
                nn.ReLU(),
                nn.Linear(n_hidden, n_output),
            )
        else: 
            print("Error! the depth is not in 3-10")
    
    #定义前向运算
    def forward(self, x):
        k = self.net(x)
        return k



def GPUstrain2(x, y, x_valid, y_valid, args,seed,nocuda):  
    """
    Train a neural network on GPU or CPU based on the given configuration, using early stopping.
    
    Parameters:
    x: Training input data.
    y: Training target data.
    x_valid: Validation input data.
    y_valid: Validation target data.
    x_test: Test input data.
    y_test: Test target data.
    args: Arguments object containing hyperparameters.
    seed: Seed and identifier.
    nocuda (int): Flag to select device; options include specific GPU IDs, CPU, or auto-selection.
    
    Returns:
    tuple: (trained network, list of training losses, list of validation losses, list of test errors)
    """

    x_dim = 1 

    if nocuda == 0:
        device = torch.device("cuda:0")
    if nocuda == 1:
        device = torch.device("cuda:1")
    if nocuda == 100:
        device = get_free_gpu()
    if nocuda == -1:
        device = torch.device("cpu")


    net = happynet(n_feature=x_dim, n_hidden=args.wide, n_output=1, n_layer=args.depth).to(device)
    nepoch = args.nepoch
    
    optimizer=torch.optim.Adam(net.parameters(), lr=args.lr, betas=(0.90, 0.999), eps=1e-8, weight_decay=0., amsgrad=False,)  
    loss_func=nn.MSELoss() 
    train_epochs_loss = [] 
    valid_epochs_loss = [] 
    x = x.reshape(-1,x_dim)
    y = y.reshape(-1)

    x=torch.from_numpy(x).float().to(device)
    y=torch.from_numpy(y).float().to(device)
    x_valid=torch.from_numpy(x_valid).float().to(device) 
    y_valid=torch.from_numpy(y_valid).float().to(device) 
    train_dataset = Dataset_repeatedmeasurement(x,y)
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=args.batch_size, shuffle=True)

    save_path = "./resultsv" 
    early_stopping = EarlyStopping(save_path,args=args)

    for epoch in range(nepoch): 
        net.train()
        train_epoch_loss = []

        # =========================train=========================
        for idx, traindata in enumerate(train_dataloader):
            x_train, y_train = traindata
            x_train=torch.Tensor(x_train).float().view(-1,1,x_dim)
            y_train=torch.Tensor(y_train).float()
            outputs=net(x_train) 

            loss=loss_func(outputs.view(-1),y_train.view(-1).float())

            optimizer.zero_grad() 

            loss.backward()  
            optimizer.step()  
            
            train_epoch_loss.append(loss.item())

        del outputs, loss

        train_epochs_loss.append(np.average(train_epoch_loss))

        # =========================valid=========================
        with torch.no_grad():
            net.eval() 

            valid_predict=net(x_valid.view(-1,1,x_dim))
            valid_y_pre=valid_predict.view(-1).detach()
            valid_y_pre=torch.Tensor(valid_y_pre).float()
            loss_valid=loss_func(valid_y_pre, y_valid.view(-1).float())
            valid_epochs_loss.append(loss_valid.item())



        print("epoch = {}, training loss = {}, validation loss = {}".format(epoch, np.average(train_epoch_loss), loss_valid)) 


        if epoch > 10 or args.n_train*args.m_train < 200:
            early_stopping(net, np.average(train_epoch_loss), loss_valid, args,seed)
            if early_stopping.early_stop: 
                print("Early stopping")
                break  


        del valid_predict, valid_y_pre, loss_valid 
    gc.collect()

    return net, train_epochs_loss, valid_epochs_loss  



torch.manual_seed(12345) 
np.random.seed(12345) 
random.seed(12345) 
torch.cuda.manual_seed_all(12345) 

robjects.r['load']("./data/datahao/data2008.Rdata") 

nnn_vec = [100,500,1000,5000]
mma = [1,5,10,20,30,25,50]
res = np.zeros(shape=(len(nnn_vec), len(mma), 3))

for nnnind in range(len(nnn_vec)):
    nnn = nnn_vec[nnnind]
    n_train = nnn
    n_valid = math.ceil(nnn * 0.25)
    torch.manual_seed(123456) 
    np.random.seed(123456) 
    random.seed(123456) 
    torch.cuda.manual_seed_all(123456) 
    randind = list(range(len(robjects.r['data'][0])))
    random.shuffle(randind)
    train_ind = [randind[i] for i in range(0, math.ceil(len(robjects.r['data'][0]) * 0.8 ))]
    valid_ind = [randind[i] for i in range(math.ceil(len(robjects.r['data'][0]) * 0.8 ), len(robjects.r['data'][0]))]
    train_ind = [train_ind[i] for i in range(n_train)]
    valid_ind = [valid_ind[i] for i in range(n_valid)]

    for mind in range(len(mma)):
        m = mma[mind]
        m_train = m
        m_valid = m
        y_train = np.array([]) 
        pp_train = np.array([]) 
        for i in train_ind:
            random.seed(123*i) 
            train_ind_m = list(range(50))
            random.shuffle(train_ind_m)
            for jj in range(m):
                j = train_ind_m[jj]
                y_train = np.append(y_train, robjects.r['data'][1][i][j])
                pp_train = np.append(pp_train, robjects.r['data'][0][i][j])
                y_train = y_train.reshape(-1,1)
                pp_train = pp_train.reshape(-1,1)

        y_valid = np.array([]) 
        pp_valid = np.array([]) 
        for i in valid_ind:
            random.seed(123*i*6) 
            train_ind_m = list(range(50))
            random.shuffle(train_ind_m)
            for jj in range(m):
                j = train_ind_m[jj]
                y_valid = np.append(y_valid, robjects.r['data'][1][i][j])
                pp_valid = np.append(pp_valid, robjects.r['data'][0][i][j])
                y_valid = y_valid.reshape(-1,1)
                pp_valid = pp_valid.reshape(-1,1)

        if n_train*m_train < 128:
            batch_size= min(n_train*m_train, 32)
            lr = 0.0005
        elif n_train*m_train < 1024:
            batch_size= 64
            lr = 0.0005
        elif n_train*m_train < 4096:
            batch_size= 128
            lr = 0.001
        elif n_train*m_train < 8192:
            batch_size= 256
            lr = 0.001
        elif n_train*m_train < 16384:
            batch_size= 512
            lr = 0.002
        else:
            batch_size= 1024
            lr = 0.002
        nocuda = 0
        trun = 30
        torch.manual_seed(123) 
        np.random.seed(123) 
        random.seed(123) 
        torch.cuda.manual_seed_all(123) 

        args = Args(lr=lr, wide=50, depth = 2, batch_size= batch_size, n_train=n_train, m_train=m_train)
        GPUstrain2(x=pp_train,y=y_train,x_valid = pp_valid,y_valid=y_valid, args=args,seed = 123, nocuda = nocuda)

        a = torch.load('./resultsv/best'+ str(123) + args.biaoji +'train_loss.pth')
        b = torch.load('./resultsv/best'+ str(123) + args.biaoji +'valid_loss.pth')
        net0 = torch.load('./resultsv/best'+ str(123) + args.biaoji +'network.pth')
        a = np.expand_dims(a , 0)
        b = np.expand_dims(b.cpu(), 0)
        c0 = np.r_[a,b,0]

        args = Args(lr=lr, wide=100, depth = 3, batch_size= batch_size, n_train=n_train, m_train=m_train)
        GPUstrain2(x=pp_train,y=y_train,x_valid = pp_valid,y_valid=y_valid, args=args,seed = 123, nocuda = nocuda)
        a = torch.load('./resultsv/best'+ str(123) + args.biaoji +'train_loss.pth')
        b = torch.load('./resultsv/best'+ str(123) + args.biaoji +'valid_loss.pth')
        net1 = torch.load('./resultsv/best'+ str(123) + args.biaoji +'network.pth')
        a = np.expand_dims(a , 0)
        b = np.expand_dims(b.cpu(), 0)
        c1 = np.r_[a,b,1]


        args = Args(lr=lr, wide=200, depth = 4, batch_size= batch_size, n_train=n_train, m_train=m_train)
        GPUstrain2(x=pp_train,y=y_train,x_valid = pp_valid,y_valid=y_valid, args=args,seed = 123, nocuda = nocuda)
        a = torch.load('./resultsv/best'+ str(123) + args.biaoji +'train_loss.pth')
        b = torch.load('./resultsv/best'+ str(123) + args.biaoji +'valid_loss.pth')
        net2 = torch.load('./resultsv/best'+ str(123) + args.biaoji +'network.pth')
        a = np.expand_dims(a , 0)
        b = np.expand_dims(b.cpu(), 0)
        c2 = np.r_[a,b,2]

        args = Args(lr=lr, wide=400, depth = 5, batch_size= batch_size, n_train=n_train, m_train=m_train)
        GPUstrain2(x=pp_train,y=y_train,x_valid = pp_valid,y_valid=y_valid, args=args,seed = 123, nocuda = nocuda)
        a = torch.load('./resultsv/best'+ str(123) + args.biaoji +'train_loss.pth')
        b = torch.load('./resultsv/best'+ str(123) + args.biaoji +'valid_loss.pth')
        net3 = torch.load('./resultsv/best'+ str(123) + args.biaoji +'network.pth')
        a = np.expand_dims(a , 0)
        b = np.expand_dims(b.cpu(), 0)
        c3 = np.r_[a,b,3]

        args = Args(lr=lr, wide=600, depth = 6, batch_size= batch_size, n_train=n_train, m_train=m_train)
        GPUstrain2(x=pp_train,y=y_train,x_valid = pp_valid,y_valid=y_valid, args=args,seed = 123, nocuda = nocuda)
        a = torch.load('./resultsv/best'+ str(123) + args.biaoji +'train_loss.pth')
        b = torch.load('./resultsv/best'+ str(123) + args.biaoji +'valid_loss.pth')
        net4 = torch.load('./resultsv/best'+ str(123) + args.biaoji +'network.pth')
        a = np.expand_dims(a , 0)
        b = np.expand_dims(b.cpu(), 0)
        c4 = np.r_[a,b,4]
        
        args = Args(lr=lr, wide=800, depth = 6, batch_size= batch_size, n_train=n_train, m_train=m_train)
        GPUstrain2(x=pp_train,y=y_train,x_valid = pp_valid,y_valid=y_valid, args=args,seed = 123, nocuda = nocuda)
        a = torch.load('./resultsv/best'+ str(123) + args.biaoji +'train_loss.pth')
        b = torch.load('./resultsv/best'+ str(123) + args.biaoji +'valid_loss.pth')
        net5 = torch.load('./resultsv/best'+ str(123) + args.biaoji +'network.pth')
        a = np.expand_dims(a , 0)
        b = np.expand_dims(b.cpu(), 0)
        c5 = np.r_[a,b,5]

        p = np.r_[np.expand_dims(c0, 0),np.expand_dims(c1, 0),np.expand_dims(c2, 0),np.expand_dims(c3, 0),np.expand_dims(c4, 0),np.expand_dims(c5, 0)]
        ind = np.argmin(p[:,1])
        res[nnnind,mind,:]=p[ind]
        torch.save( eval("net"+str(ind)) , os.path.join("./bestnet/", 'best' + str(nnn) +"m"+ str(m) +'net.pth') )
